# Post Here: Subreddit Predictor

## Recommendation API - 1.2

> aka: the MVP Classifier®

---
---

## Intro - MVP Classifier (Model #2)

The second iteration of the model for recommending (predicting) appropriate subreddit(s) will be built using a somewhat naive approach to text analysis and a multi-class classification model.

The model will be trained using the [reddit self-post classification task dataset](https://www.kaggle.com/mswarbrickjones/reddit-selfposts), available on Kaggle thanks to [Evolution AI](https://evolution.ai//blog/page/5/an-imagenet-like-text-classification-task-based-on-reddit-posts/).

The full dataset includes 1,013,000 rows (1000 records each from 1013 subreddits). To keep things manageable for the proof-of-concept, the first 10,000 records will be used to train the embeddings (vectorizer) and models.

According to the dataset description on Kaggle, the data has already been randomized. Therefore, reading in the first 10k records, as opposed to extracting a random sample, will not introduce any more bias into the data.

---

### Imports

In [1]:
# === General imports === #
import pandas as pd
import numpy as np
import os

In [2]:
# === sklearn imports === #
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

---

### Load and preprocess the data

In [42]:
# === Load the dataset === #
# import pandas as pd

# This time only first 10k rows
df1 = pd.read_csv("rspct_100k.csv", sep="\t", nrows=10000)

# === First looks === #
print(df1.shape)
df1.head()

(10000, 4)


,id,subreddit,title,selftext
0,0,talesfromtechsupport,Remember your command line switches...,"Hi there, <lb>The usual. Long time lerker, fi..."
1,1,teenmom,"So what was Matt ""addicted"" to?",Did he ever say what his addiction was or is h...
2,2,Harley,No Club Colors,Funny story. I went to college in Las Vegas. T...
3,3,ringdoorbell,"Not door bell, but floodlight mount height.",I know this is a sub for the 'Ring Doorbell' b...
4,4,intel,Worried about my 8700k small fft/data stress r...,"Prime95 (regardless of version) and OCCT both,..."


In [43]:
# === Save the 10k records to new csv === #
df1.to_csv("../../assets/rspct_10k.csv", sep="\t", index=False)

In [2]:
# === Load the saved version === #
df1 = pd.read_csv("../assets/rspct_10k.csv", sep="\t")

# === First looks === #
print(df1.shape)
df1.head()

(10000, 4)


,id,subreddit,title,selftext
0,0,talesfromtechsupport,Remember your command line switches...,"Hi there, <lb>The usual. Long time lerker, fi..."
1,1,teenmom,"So what was Matt ""addicted"" to?",Did he ever say what his addiction was or is h...
2,2,Harley,No Club Colors,Funny story. I went to college in Las Vegas. T...
3,3,ringdoorbell,"Not door bell, but floodlight mount height.",I know this is a sub for the 'Ring Doorbell' b...
4,4,intel,Worried about my 8700k small fft/data stress r...,"Prime95 (regardless of version) and OCCT both,..."


In [4]:
# === Split up dataset into train and test === #
# from sklearn.model_selection import train_test_split

# 80% train, 20% test, stratified on the target
train, test = train_test_split(df1, test_size=0.2, stratify=df1["subreddit"])

train.shape, test.shape

((8000, 4), (2000, 4))

In [5]:
# === Arrange data into feature and target === #

# MVP model only uses 'selftext' feature
X_train = train["selftext"]
X_test = test["selftext"]

# Predict the subreddit of each post
y_train = train["subreddit"]
y_test = test["subreddit"]

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(8000,) (2000,)
(8000,) (2000,)


In [6]:
# === Encode the target using LabelEncoder === #

# This process naively transforms each class of the target into a number
# from sklearn.preprocessing import LabelEncoder

le = LabelEncoder() # Instantiate a new encoder instance
le.fit(y_train)  # Fit it on training label data

# Transform both using the train-fit instance
y_train = le.transform(y_train)
y_test  = le.transform(y_test)

y_train[:8]

array([251, 999, 570, 233, 544, 433, 467, 377])

---
---

### Vectorization

The vectorizer that will be used to convert the words into numbers will not analyze the text for meaning or anything like that - remember, this is the MVP model. We can get as crazy as we want after we have a working baseline.

TF-IDF vectorization finds the unique aspects of documents of text, based on a simple count of the words within each document. In this context, "document" refers to an individual reddit post.

The vectorizer can be instantiated then "trained" on the dataset. One way to think about the training step is that it builds a vectorized vocabulary of the words in the dataset.

The TF-IDF implementation that will be used in the MVP comes from scikit-learn:

> [sklearn.feature_extraction.text.TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

A custom tokenizer function can be passed into the vectoirizer to increase the quality of the tokens (number representations of words). The tokenizer function will use the NLP library [spaCy](https://spacy.io/usage/).

However, for the MVP, we will go with the built-in tokenizer function built into the TfidfVectorizer class.

In [25]:
# === Vectorize! === #

# Extract features from the text data using bag-of-words (single words + bigrams).
# Uses tfidf weighting (helps a little for Naive Bayes in general).
# from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    max_features=20000,
    min_df=5,
    ngram_range=(1,2),
    stop_words=None,  # TODO: try out spacy's or sklearn's stopwords
)

# Fit the vectorizer on the feature column to create vocab (doc-term matrix)
# This process is split into component parts (fit / transform) for pickling
vocab = tfidf.fit(X_train)

# Get sparse document-term matrices
X_train_sparse = vocab.transform(X_train)
X_test_sparse = vocab.transform(X_test)

In [26]:
# === Naive Bayes model === #
# from sklearn.naive_bayes import MultinomialNB

# Instantiate and train the model
nb = MultinomialNB(alpha=0.1)
nb.fit(X_train_sparse, y_train)

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [27]:
# === Create predictions on test feature === #
y_pred_proba = nb.predict_proba(X_test_sparse)

print(y_pred_proba.shape)

(2000, 1013)


In [28]:
# === For each prediction, find the index with the highest probability === #
# import numpy as np
y_pred = np.argmax(y_pred_proba, axis=1)
y_pred[:10]

array([279, 279, 279, 279, 598, 560, 185, 553, 185, 185])

In [29]:
# === Evaluate performance using precision-at-k === #

def precision_at_k(y_true, y_pred, k=5):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_pred = np.argsort(y_pred, axis=1)
    y_pred = y_pred[:, ::-1][:, :k]
    arr = [y in s for y, s in zip(y_true, y_pred)]
    return np.mean(arr)

print('precision@1 =', np.mean(y_test == y_pred))
print('precision@3 =', precision_at_k(y_test, y_pred_proba, 3))
print('precision@5 =', precision_at_k(y_test, y_pred_proba, 5))

# 1,000,000 records
# precision@1 = 0.6732724580454097
# precision@3 = 0.8058588351431392
# precision@5 = 0.8481391905231984

# 100,000 records
# precision@1 = 0.45935
# precision@3 = 0.6075
# precision@5 = 0.665

# 10,000 records
# precision@1 = 0.108
# precision@3 = 0.1785
# precision@5 = 0.215

precision@1 = 0.108
precision@3 = 0.1785
precision@5 = 0.215


---

### Predict subreddit from new input

Now that our model is trained and we have our baseline, we can use it to predict what subreddit would belong to a new piece of data (a post).

In [12]:
# === Example post from 'r/learnprogramming' === #

post = """I am a new grad looking for a job and currently in the process with a company for a junior backend engineer role. I was under the impression that the position was Javascript but instead it is actually Java. My general programming and "leet code" skills are pretty good, but my understanding of Java is pretty shallow. How can I use the next three days to best improve my general Java knowledge? Most resources on the web seem to be targeting complete beginners. Maybe a book I can skim through in the next few days?

Edit:

A lot of people are saying "the company is a sinking ship don't even go to the interview". I just want to add that the position was always for a "junior backend engineer". This company uses multiple languages and the recruiter just told me the incorrect language for the specific team I'm interviewing for. I'm sure they're mainly interested in seeing my understanding of good backend principles and software design, it's not a senior lead Java position."""

In [23]:
# === Function to serve predictions === #
# The main functionality of the predict API endpoint

def predict(post: str, n: int = 5) -> dict:
    """
    Serve subreddit predictions.
    
    Parameters
    ----------
    post : string
        Selftext that needs a home.
    n    : integer
        The desired name of the output file,
        not including the '.pkl' extension.

    Returns
    -------
    Python dictionary formatted as follows:
        [{'subreddit': 'PLC', 'proba': 0.014454},
         ...
         {'subreddit': 'Rowing', 'proba': 0.005206}]
    """
    
    # Vectorize the post -> sparse doc-term matrix
    post_vec = vocab.transform([post])
    
    # Generate predicted probabilities from trained model
    proba = nb.predict_proba(post_vec)
    
    # Wrangle into correct format
    return (pd
                .DataFrame(proba, columns=[le.classes_])  # Classes as column names
                .T  # Transpose so column names become index
                .reset_index()  # Pull out index into a column
                .rename(columns={"level_0": "subreddit", 0: "proba"})  # Rename for aesthetics
                .sort_values(by="proba", ascending=False)  # Sort by probability
                .iloc[:n]  # n-top predictions to serve
                .to_dict(orient="records")
               )

In [24]:
# === Test out the function === #
post_pred = predict(post)  # Default is 5 results
post_pred

[{'subreddit': 'PLC', 'proba': 0.014454003455867464},
 {'subreddit': 'sales', 'proba': 0.008377114822879663},
 {'subreddit': 'AskHR', 'proba': 0.007373675677628392},
 {'subreddit': 'OccupationalTherapy', 'proba': 0.00561548847793111},
 {'subreddit': 'Rowing', 'proba': 0.005206212277479405}]

In [30]:
# === Test it out with another dummy post === #

# This one comes from r/suggestmeabook
post2 = """I've been dreaming about writing my own stort story for a while but I want to give it an unexpected ending. I've read lots of books, but none of them had the plot twist I want. I want to read books with the best plot twists, so that I can analyze what makes a good plot twist and write my own story based on that points. I don't like romance novels and I mostly enjoy sci-fi or historical books but anything beside romance novels would work for me, it doesn't have to be my type of novel. I'm open to experience after all. I need your help guys. Thanks in advance."""

In [31]:
# === This time with 10 results === #
post2_pred = predict(post2, n=10)
post2_pred

[{'subreddit': 'PLC', 'proba': 0.008913827946281429},
 {'subreddit': 'JUSTNOMIL', 'proba': 0.005364412871980992},
 {'subreddit': 'vaginismus', 'proba': 0.005343515334435943},
 {'subreddit': 'Rowing', 'proba': 0.004869320257487137},
 {'subreddit': 'hookah', 'proba': 0.00434668954125044},
 {'subreddit': 'dresdenfiles', 'proba': 0.004295750109887567},
 {'subreddit': 'StudentLoans', 'proba': 0.004077573292318194},
 {'subreddit': 'productivity', 'proba': 0.004050498345338448},
 {'subreddit': 'flexibility', 'proba': 0.003925624264099203},
 {'subreddit': 'dpdr', 'proba': 0.003923252551916036}]

> _Terrible results. But hey, it works!_

---

### Pickle Time

In [32]:
# === Create pickle func to make pickling (a little) easier === #

def picklizer(to_pickle, filename, path):
    """
    Creates a pickle file.
    
    Parameters
    ----------
    to_pickle : Python object
        The trained / fitted instance of the 
        transformer or model to be pickled.
    filename : string
        The desired name of the output file,
        not including the '.pkl' extension.
    path : string or path-like object
        The path to the desired output directory.
    """
    import os
    import pickle

    # Create the path to save location
    picklepath = os.path.join(path, filename)

    # Use context manager to open file
    with open(picklepath, "wb") as p:
        pickle.dump(to_pickle, p)

In [33]:
# === Picklize! === #
filepath = "../assets"  # Change this accordingly

# Export vectorizer as pickle
picklizer(vocab, "02_vocab.pkl", filepath)

# Export naive bayes model as pickle
picklizer(nb, "02_nb.pkl", filepath)

Load and consume pickles...

> _Enjoy responsibly._

In [34]:
# === Load the trained vectorizer === #
import pickle

vocab_path = os.path.join(filepath, "02_vocab.pkl")

# Use context manager to open and load pickle
with open(vocab_path, "rb") as p:
    vocab = pickle.load(p)

In [35]:
# === Load the somewhat-trained Naive Bayes classifier === #
# import pickle

nb_path = os.path.join(filepath, "02_nb.pkl")

# Use context manager to open and load pickle
with open(nb_path, "rb") as p:
    nb = pickle.load(p)

In [37]:
# === Test out the pickled versions === #

# This one comes from r/buildmeapc
post3 = """I posted my wants for my build about 2 months ago. Ordered them and when I went to build it I was soooooo lost. It took 3 days to put things together because I was afraid I would break something when I finally got the parts together it wouldn’t start, I was so defeated. With virtually replacing everything yesterday it finally booted and I couldn’t be more excited!"""

# This time I'll pass in 20, because why not?
post3_recs = predict(post3, 20)
post3_recs

[{'subreddit': 'JUSTNOMIL', 'proba': 0.011555053885259908},
 {'subreddit': 'vaginismus', 'proba': 0.008987155038573382},
 {'subreddit': 'dji', 'proba': 0.00841345865980121},
 {'subreddit': 'benzodiazepines', 'proba': 0.0066139336598462365},
 {'subreddit': 'badroommates', 'proba': 0.005933263759491765},
 {'subreddit': 'breastfeeding', 'proba': 0.00517916872820697},
 {'subreddit': 'Rowing', 'proba': 0.005096592044629702},
 {'subreddit': 'parrots', 'proba': 0.004760957552212924},
 {'subreddit': 'Miscarriage', 'proba': 0.004497028139218237},
 {'subreddit': 'Allergies', 'proba': 0.0041798669967584865},
 {'subreddit': 'TryingForABaby', 'proba': 0.004125437914256018},
 {'subreddit': 'speedrun', 'proba': 0.004122056949710342},
 {'subreddit': 'OccupationalTherapy', 'proba': 0.003930039884282085},
 {'subreddit': 'Throwers', 'proba': 0.003821378766304194},
 {'subreddit': 'AvPD', 'proba': 0.0037124339868972564},
 {'subreddit': 'lego', 'proba': 0.003607223965090716},
 {'subreddit': 'adderall', 'pro